In [ ]:
import numpy as np
import pandas as pd
import random

pm=pd.read_csv("propability_matrix.csv")     
pm.set_index("location",inplace=True)
        
PROBABILITIE_MATRIX=pm



class Customer:
     
    def __init__(self, entry_time,custid,probability_matrix=pm):
        self.location = "entry"
        self.probability_matrix = probability_matrix
        assert type(entry_time) == pd._libs.tslibs.timestamps.Timestamp , "entry time has to be pandas.Timestamp"
        self.current_time = entry_time
        self.cust_id=custid    
        
    def do_next_move(self): 
        locations=list(pm.columns)
        probabilities=pm.loc[self.location]
        if self.location != "entry":
             self.current_time = self.current_time + pd.Timedelta("60s")
        self.location=random.choices(locations,weights=list(probabilities))[0]
        
    def do_shopping(self):
        self.track=[self.location,]
        self.timings=[self.current_time,]
        while self.location !="checkout":
            self.do_next_move() 
            if self.current_time.hour >= 22:
                self.track.append("checkout")
                self.timings.append(self.current_time)
                break
            self.track.append(self.location)
            self.timings.append(self.current_time)      
        self.get_times()

    def get_times(self):
        hours   = [t.hour   for t in self.timings]
        minutes = [t.minute for t in self.timings]
        times=[str(h).zfill(2) + ":"+ str(minutes[i]).zfill(2) for i,h in enumerate(hours)]
        self.times=times
    


In [ ]:
import pylab as plt

In [ ]:
cust=Customer(pd.Timestamp(2020,1,1,12,59,0),1)
cust.do_shopping()
plt.plot(cust.times, cust.track, "o")
plt.plot(cust.times, cust.track, "--")

In [ ]:
pm

In [ ]:
class SuperMarket:
    def __init__(self):
        self.start_hour=8
        self.stop_hour=22
        self.year=2020
        self.month=1
        self.day=1
        self.current_time=pd.Timestamp(self.year,self.month,self.day,self.start_hour,0,0)
        #self.nocust=poisson(1.0, 1)
        
    def number_of_customers(self):
        from numpy.random import poisson
        self.newcust_per_min=poisson(1.0, 1)[0] # [0] to get frist element of the array
    
    def run_day_simulator(self):
        cust_cnt=0
        self.cust_dict={}
        while self.current_time < pd.Timestamp(self.year,self.month,self.day,self.stop_hour,0,0):
            self.number_of_customers()
            print(self.current_time)
            for _ in range(self.newcust_per_min):
                cust_cnt+=1
                cust = Customer(self.current_time, cust_cnt)
                cust.do_shopping()
                self.cust_dict[cust.cust_id]=cust.track
                print(cust.cust_id)
            self.current_time=self.current_time + pd.Timedelta("60s")
  
            
        
     

In [ ]:
SM=SuperMarket()

In [ ]:
SM.current_time

In [ ]:
SM.run_day_simulator()

In [ ]:
SM.cust_dict